In [24]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MD17
from torch_geometric.nn import GATConv, global_add_pool
from torch_geometric.nn import GATv2Conv
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#from torch_geometric.transforms import KNNGraph
from torch_geometric.transforms import RadiusGraph
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from tqdm import tqdm

In [2]:
# Compute edge indices
#knn_graph = KNNGraph(k=6) 

In [3]:
# Load the MD17 dataset
#https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/md17.html#MD17
path = "/home/sire/phd/srz228573/benchmarking_datasets/baseline_model_dataset/gat"
#dataset = MD17(root=path, name = 'revised benzene', transform=KNNGraph(k=6))
dataset = MD17(root=path, name = 'revised benzene', transform= RadiusGraph(r=2))

In [65]:
dataset.print_summary()

100%|██████████| 100000/100000 [00:24<00:00, 4148.72it/s]

MD17 (#graphs=100000):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |       12 |     24   |
| std        |        0 |      0.2 |
| min        |       12 |     24   |
| quantile25 |       12 |     24   |
| median     |       12 |     24   |
| quantile75 |       12 |     24   |
| max        |       12 |     28   |
+------------+----------+----------+


In [29]:
dataset[0]

Data(pos=[12, 3], z=[12], energy=[1], force=[12, 3], edge_index=[2, 24])

In [77]:
# # Prepare the data
# for data in dataset:
#     data.z = data.z.unsqueeze(-1).float()  # Use atomic numbers as node features


In [4]:
# Use 80% of the data for training, and 20% for testing
train_dataset = dataset[:int(0.8 * len(dataset))]
test_dataset = dataset[int(0.8 * len(dataset)):]

In [5]:
train_dataset[0]

Data(pos=[12, 3], z=[12], energy=[1], force=[12, 3], edge_index=[2, 24])

In [6]:

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

/home/sire/phd/srz228573/anaconda3/envs/bebam/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [15]:
# class GAT(torch.nn.Module):
#     def __init__(self, num_features, hidden_channels, num_classes):
#         super(GAT, self).__init__()
#         self.conv1 = GATConv(num_features, hidden_channels)
#         self.conv2 = GATConv(hidden_channels, hidden_channels)
#         self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
#         self.fc2 = torch.nn.Linear(hidden_channels, num_classes)

#     def forward(self, x, edge_index, batch):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = F.dropout(x, p=0.6, training=self.training)
#         x = self.conv2(x, edge_index)
#         x = F.relu(x)
#         x = global_add_pool(x, batch)  # Pooling to obtain graph-level representation
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         return x
class GAT(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GAT, self).__init__()
        self.embedding = torch.nn.Embedding(num_features, hidden_channels)
        self.conv1 = GATv2Conv(hidden_channels, hidden_channels)
        self.conv2 = GATv2Conv(hidden_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, z, edge_index, batch):
        #print("inside_forward")
        x = self.embedding(z)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_add_pool(x, batch)  # Pooling to obtain graph-level representation
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [16]:
#num_features = max([data.z.max().item() for data in dataset]) - 1 + 2
num_features = 7


In [17]:

model = GAT(num_features, hidden_channels=64, num_classes=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)


In [25]:
def train():
    model.train()
    for data in tqdm(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.z, data.edge_index, data.batch)
        loss = F.mse_loss(output, data.energy)
        loss.backward()
        optimizer.step()
        print(loss.item())


In [26]:
def test(loader):
    model.eval()
    loss = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            output = model(data.z, data.edge_index, data.batch)
        loss += F.mse_loss(output, data.energy).item()
    return loss / len(loader)

In [27]:
for epoch in range(2):
    train()
    train_loss = test(train_loader)
    test_loss = test(test_loader)
    print(f'Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

  0%|          | 0/2500 [00:00<?, ?it/s]/tmp/ipykernel_20700/1564437284.py:7: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output, data.energy)
  1%|          | 16/2500 [00:00<00:33, 75.26it/s]

76126672.0
63429720.0
73283264.0
62057356.0
78749248.0
43225416.0
50657884.0
106223248.0
68042096.0
93731240.0
81159296.0
96039168.0
64975632.0
62031152.0
81222592.0
98196040.0


  1%|▏         | 32/2500 [00:00<00:32, 76.47it/s]

65133992.0
65976296.0
74943760.0
45755300.0
57412532.0
59149832.0
69477976.0
78662024.0
53990172.0
98724920.0
67183040.0
60354484.0
65485008.0
69115168.0
48870360.0
70392096.0


  2%|▏         | 49/2500 [00:00<00:31, 78.64it/s]

...

 99%|█████████▉| 2484/2500 [00:28<00:00, 87.56it/s]

9070704.0
9371662.0


100%|██████████| 2500/2500 [00:28<00:00, 86.83it/s]

16337590.0
11862210.0
11453648.0
12147162.0
31824314.0
7647021.0
25542084.0
22149692.0
14165288.0
16238430.0
39681080.0
17329832.0
20541764.0
14654788.0
10873334.0
10310601.0


Epoch: 2, Train Loss: 1856032640.8704, Test Loss: 1856030204.7232
